This code extracts titles from the pages of results at this url:
http://www.imdb.com/list/ls057823854/

In [1]:
import urllib2
import re
import json

from bs4 import BeautifulSoup

In [2]:
################################################
### helper functions for the document below
################################################

hdr = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Accept-Encoding': 'none',
    'Accept-Language': 'en-US,en;q=0.8',
    'Connection': 'keep-alive'
}


def extractHtmlAndSoup(url,hdr):
    req1 = urllib2.Request(url, headers=hdr)
    page = urllib2.urlopen(req1)
    content = page.read()
    soup = BeautifulSoup(content)
    return soup


def extractMovieDetails(div):
    obj = {}
    
    # link to the movie page
    obj["link"] = div.find("div",attrs={"class","info"}). find("a")["href"]
    obj["name"] = div.find("div",attrs={"class","info"}).find("a").get_text()
    obj["year"] = div.find("span",attrs={"class":"year_type"}).get_text()
    obj["description"] = div.find("div",attrs={"class":"item_description"}).get_text()
    
    # We need to account for the fact that some movies don't have the director or stars listed.
    div_secondary = div.findAll("div", attrs={"class":"secondary"})
    if len(div_secondary) > 0:
        if 'Director' in div_secondary[0].get_text():
            obj["director"] = div_secondary[0].get_text().replace("Director:","").rstrip().lstrip()
        elif 'Stars' in div_secondary[0].get_text():
            obj["stars"] = div_secondary[0].get_text().replace("Stars:","").rstrip().lstrip()
        if len(div_secondary) > 1:
            if 'Director' in div_secondary[1].get_text():
                obj["director"] = div_secondary[1].get_text().replace("Director:","").rstrip().lstrip()
            elif 'Stars' in div_secondary[1].get_text():
                obj["stars"] = div_secondary[1].get_text().replace("Stars:","").rstrip().lstrip()
    
    obj["poster_img"] = div.find("div",attrs={"class":"image"}).find("img")["src"]
    obj["poster_desc"] = div.find("div",attrs={"class":"image"}).find("img")["alt"]
    #
    return obj

################################################

In [3]:
# url = "http://www.imdb.com/list/ls057823854/?start=101&view=detail&sort=listorian:asc"
start_num = 1

url = "http://www.imdb.com/list/ls057823854/?start=" + str(start_num) + "&view=detail&sort=listorian:asc"

page_soup = extractHtmlAndSoup(url,hdr)

/Users/luuk/pyEnv/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [4]:
# a quick inspection of the page source reveals that to extract the movie name, 
# link and date we need the content of divs with class="info"

movies_blocks = page_soup.findAll("div", attrs={"class":"list_item"})

In [5]:
MOVIES = []

for movie_block in movies_blocks:
    MOVIES.append(extractMovieDetails(movie_block))

print len(MOVIES)
print json.dumps(MOVIES[:2], indent=4)

100
[
    {
        "poster_desc": "Image of Pulp Fiction", 
        "description": "The lives of two mob hit men, a boxer, a gangster's wife, and a pair of diner bandits intertwine in four tales of violence and redemption. (154 mins.)", 
        "director": "Quentin Tarantino", 
        "link": "/title/tt0110912/", 
        "stars": "John Travolta, Uma Thurman, Samuel L. Jackson, Bruce Willis", 
        "year": "(1994)", 
        "poster_img": "http://ia.media-imdb.com/images/M/MV5BMTkxMTA5OTAzMl5BMl5BanBnXkFtZTgwNjA5MDc3NjE@._V1._SY209_CR0,0,140,209_.jpg", 
        "name": "Pulp Fiction"
    }, 
    {
        "poster_desc": "Image of The Amazing Spider-Man 2", 
        "description": "When New York is put under siege by Oscorp, it is up to Spider-Man to save the city he swore to protect as well as his loved ones. (142 mins.)", 
        "director": "Marc Webb", 
        "link": "/title/tt1872181/", 
        "stars": "Andrew Garfield, Emma Stone, Jamie Foxx, Paul Giamatti", 
        "y

In [6]:
num_movies = 9969

for i in range(2, (num_movies/100)+1):
    
    start_from = str(i * 100 + 1)
    print start_from, '.',
    
    # Create the url.
    url = "http://www.imdb.com/list/ls057823854/?start=" + start_from + "&view=detail&sort=listorian:asc"
    
    # Soup the page contents
    page_soup = extractHtmlAndSoup(url,hdr)
    
    # Get all divs that hold movies.
    movies_blocks = page_soup.findAll("div",attrs={"class":"list_item"})
    
    # Grab the movie details from each of the movies.
    for movie_block in movies_blocks:
        MOVIES.append(extractMovieDetails(movie_block))


print '\n\nDONE!'

201 . 301 . 401 . 501 . 601 . 701 . 801 . 901 . 1001 . 1101 . 1201 . 1301 . 1401 . 1501 . 1601 . 1701 . 1801 . 1901 . 2001 . 2101 . 2201 . 2301 . 2401 . 2501 . 2601 . 2701 . 2801 . 2901 . 3001 . 3101 . 3201 . 3301 . 3401 . 3501 . 3601 . 3701 . 3801 . 3901 . 4001 . 4101 . 4201 . 4301 . 4401 . 4501 .

AttributeError: 'NoneType' object has no attribute 'get_text'

In [9]:
# Just to visually check if things are alright...
print MOVIES[-1]['name']
print MOVIES[0]['name'], '|', MOVIES[100]['name'], '|', MOVIES[200]['name']

I, Frankenstein
Pulp Fiction | Raiders of the Lost Ark | Rio


In [73]:
# Save to file.
with open('../../list_of_movies_v2.json', 'w') as outfile:
    json.dump(MOVIES, outfile)

In [ ]:
#print movies[0].find(True,text = re.compile("Director"))
#print "Director" in str(movies[0])